<a href="https://colab.research.google.com/github/thekizoch/numerai-kizoch/blob/main/numerai_determine_trickle_in.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thekizoch/numerai-kizoch/blob/main/notebooks/numerai-determine-trickle-in.ipynb)

In [84]:
# dependencies
!pip -q install --upgrade numerapi
from numerapi import NumerAPI
import pandas as pd
import numpy as np
import seaborn as sns

In [149]:
# initialize client
secret_key = 'use yours'
public_id = 'use yours'

napi = NumerAPI(secret_key=secret_key,
                public_id=public_id)

In [141]:
# list of model names
list_model_names = ['1_thekizoch', '1_v2_thekizoch', '3_thekizoch',]
# define weekly trickle amount in NMR
weekly_trickle = 40
# define smoothing
alpha = 0.8
# define threshold of sharpe
threshold_sharpe = 0.2
# input multipliers for all models
multiplier_corr = 1
multiplier_mmc = 2
# use sharpe ratio to weight model
risk_weighted = True
# past rounds to consider
rounds_to_use = 20

# run

In [142]:
# empty list to hold scores
l = []
# empty list to hold sharpe ratio of weekly returns
l_sharpe = []

# load each model from list
for model_name in list_model_names:
  # load a single model
  print(f'loading {model_name}')
  dict_data = napi.round_model_performances(model_name)[0:rounds_to_use+3]
  df = pd.DataFrame.from_dict(dict_data)

  # keep essential rows for performance
  df = pd.DataFrame.from_dict(dict_data)
  df = df.loc[:,['corr','mmc','roundResolved','roundResolveTime']]
  # drop unresolved rounds, reorder
  df.drop(list(df.loc[df.roundResolved == False].index), inplace=True)
  df.sort_values(by='roundResolveTime', inplace=True)
  df.set_index('roundResolveTime', inplace=True)

  # add payout multipliers
  df['sum_corr_mmc_with_multipliers'] = df['corr']*multiplier_corr + df['mmc']*multiplier_mmc
  # checks that there are more than 5 resolved rounds
  if df.sum_corr_mmc_with_multipliers.notna().value_counts()[1] <= 5:
    raise ValueError(f'There are 5 or less resolved rounds for {model_name}. Remove it')


  # get smoothed score
  s = df.ewm(alpha=alpha).mean()['sum_corr_mmc_with_multipliers']

  l.append(s)
  l_sharpe.append(s.mean()/s.std())

loading 1_thekizoch
loading 1_v2_thekizoch
loading 3_thekizoch


In [143]:
# create frame of all smoothed scores
ll = []
for i, s in enumerate(l):
  s = s.to_frame().rename(columns={'sum_corr_mmc_with_multipliers': 
                                   list_model_names[i]})
  ll.append(s)
  
df = pd.concat(ll, axis=1)

## separate winners from losers

In [144]:
# gets winners above threshold
winners = [df.columns[i] for i in range(len(l_sharpe)) if l_sharpe[i] > threshold_sharpe]
# gets losers below threshold
losers = list(set(df.columns) - set(winners))

In [145]:
# define as frames
df_winners = df[winners]
df_losers = df[losers]

In [146]:
# remove losers
s_sharpe_winners = pd.Series(l_sharpe)
s_sharpe_winners = s_sharpe_winners.loc[s_sharpe_winners > threshold_sharpe]
s_sharpe_winners.index = df_winners.columns

s_sharpe_winners

1_thekizoch       1.463108
1_v2_thekizoch    0.646301
dtype: float64

In [147]:
# calculate weights of trickle from performance of last round
if risk_weighted == True:
  print('risk weighting the performance of models')
  s_weights = df_winners.iloc[-1] * s_sharpe_winners
else:
  print('not risk weighting the performance of models')
  s_weights = df_winners.iloc[-1]
  
total = s_weights.sum()

risk weighting the performance of models


## increase winner's stakes

In [ ]:
# increase stake
for model_name in winners:
  model = napi.get_models()[model_name]
  stake_increase = weekly_trickle * s_weights[model_name]/total
  print(f'{stake_increase} NMR stake increase for model {model_name}')  
  napi.stake_increase(stake_increase, model)

## TODO

- drain loser's stake
- automate stake management, weekly
- use cloud run, with weekly scheduler calling it